In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import Parameter
import torch.nn.functional as F
import math

from sklearn.datasets import fetch_20newsgroups
import numpy as np

import argparse
from types import SimpleNamespace

import random
random.seed(1234)

import pandas as pd
import matplotlib.pyplot as plt

from nltk.tokenize import RegexpTokenizer

import bsg, tools

import gensim.downloader as api
from gensim.models import Word2Vec, FastText, KeyedVectors

In [2]:
princeton_data = pd.read_csv("articles_1999.csv", delimiter="\|\~\|", header=None)
princeton_data.columns = ["ID", "headline", "date", "type", "author", "author_title", "article", "author_bio"]
princeton_data = princeton_data[princeton_data["article"].notnull()]


/anaconda3/envs/math689env/lib/python3.7/site-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


In [39]:
prince_data = list(princeton_data.loc[0:25,"article"])

In [40]:
len(prince_data)

21

In [41]:
### Stuff ###

window = 2
t = np.power(10.0,-4.0)



In [42]:
vocabulary, tokenized_corpus, word2idx, idx2word = tools.tokenize_corpus(prince_data)

indexed_corpus = tools.index_tokenized_corpus(tokenized_corpus, word2idx)

flattened_indexed_corpus = tools.flatten(indexed_corpus)
unigram_subsampling = tools.make_unigram_dict(flattened_indexed_corpus)

subsampled_indexed_corpus = tools.subsample_corpus(indexed_corpus, unigram_subsampling, t)

sum([len(doc) for doc in subsampled_indexed_corpus])

len(flattened_indexed_corpus)

corpus_center_words, corpus_context_words = tools.get_corpus_centers_contexts(subsampled_indexed_corpus, window)

unigram = tools.make_unigram_dict(corpus_center_words)

In [ ]:
## w2v ##

In [ ]:
# all_prince = list(princeton_data.loc[0:princeton_data.shape[0],6])
# vocabulary, tokenized_corpus, word2idx, idx2word = tools.tokenize_corpus(princeton_data["article"])



In [43]:
### word2vec ###

w2v = Word2Vec(sample = t, sg=1, negative=1, size=100, window=2, min_count=1, max_vocab_size=None, seed=1, workers=1)

w2v.build_vocab(tokenized_corpus)  # prepare the model vocabulary
w2v.train(tokenized_corpus, total_examples=w2v.corpus_count, epochs=50)


(329677, 607650)

In [44]:
"barbara" in w2v.wv.vocab

False

In [47]:
w2v.wv.most_similar("mental")

[('health', 0.5791363716125488),
 ('the', 0.5690140128135681),
 ('experience', 0.5158798694610596),
 ('not', 0.5035669803619385),
 ('for', 0.5034538507461548),
 ('with', 0.5011468529701233),
 ('always', 0.47424519062042236),
 ('and', 0.47265416383743286),
 ('campus', 0.4568369686603546),
 ('like', 0.45284149050712585)]

In [ ]:
# # Get corpus: 20 news groups

# newsgroups_train = fetch_20newsgroups(subset='train')

# window = 5

# vocabulary, tokenized_corpus, word2idx, idx2word = tools.tokenize_corpus(newsgroups_train.data)

# indexed_corpus = tools.index_tokenized_corpus(tokenized_corpus, word2idx)

# corpus_center_words, corpus_context_words = tools.get_corpus_centers_contexts(indexed_corpus, window)

# unigram = tools.make_unigram_dict(corpus_center_words)

In [48]:
args_dict = {"vocab_size" : len(vocabulary), "window" : window, 
             "batch_size" : 200, "optimizer" : 80, "learning_rate" : 0.0005, 
             "momentum" : 0.99, "num_epoch" : 50, "init_mult" : 1, 
             "variance" : 0.995, "start" : True, "nogpu" : True, 
             "embedding_dim" : 300, "freeze" : False}

args = SimpleNamespace(**args_dict)

In [49]:
model = bsg.BSG(args.window, unigram, args.vocab_size, input_dim = 25)
optimizer = torch.optim.Adam(model.parameters(), args.learning_rate, betas=(args.momentum, 0.999))
model = bsg.train(model, args, optimizer, corpus_center_words, corpus_context_words)



/Users/kristyn/Desktop/Python/bsg_pytorch/bsg.py:111: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  loss_epoch += loss.data[0]    # add loss to loss_epoch


Epoch 0, loss=144670.390625
Epoch 5, loss=78010.9765625
Epoch 10, loss=55961.828125
Epoch 15, loss=42342.171875
Epoch 20, loss=32400.54296875
Epoch 25, loss=25242.056640625
Epoch 30, loss=20759.263671875
Epoch 35, loss=16871.8046875
Epoch 40, loss=14552.9599609375
Epoch 45, loss=12564.931640625


In [50]:
# torch.save(model, "bsg_prince_50e")

In [ ]:
model = torch.load("bsg_prince_50e")

In [ ]:
idx2word(corpus_center_words[0])

In [51]:
types, counts = np.unique(corpus_center_words.tolist(), return_counts=True)

In [52]:
test_idx2realidx = {idx: realidx for (idx , realidx) in enumerate(types)}
test_realidx2idx = {realidx: idx for (idx , realidx) in enumerate(types)}

In [53]:
from sklearn.metrics.pairwise import cosine_similarity
cos_sim_matrix = cosine_similarity(model.type_means(torch.tensor(types)).detach().numpy(), 
                                   model.type_means(torch.tensor(types)).detach().numpy())
def n_closest_words(word, cos_sim_matrix, n):
    word_index = test_realidx2idx.get(word2idx.get(word))
    print("word_index is " + str(word_index))
    close_words_indices = np.argsort(cos_sim_matrix[word_index])[-n:]
    print("close_words_indices.shape" + str(close_words_indices.shape))
    return [idx2word.get(test_idx2realidx.get(j)) for j in close_words_indices]

In [106]:
n_closest_words("fraternity", cos_sim_matrix, 20)

word_index is 784
close_words_indices.shape(20,)


['plans',
 'swap',
 'reform',
 'says',
 'application',
 'entire',
 'interest',
 'idea',
 'objective',
 'address',
 'muhedini',
 'four',
 'outcomes',
 'money',
 'seemed',
 'organizer',
 'campaign',
 'half',
 'shawshank',
 'fraternity']

In [56]:
len(list(test_idx2realidx.keys()))

1220

In [58]:
model.type_means.weight.shape

torch.Size([3268, 100])